# Project : Custom Cohere with memory

Have you may have seen, there is no memory conversation with our previous LLM. 

Chatbots are one of the central LLM use-cases. The core features of chatbots are that they can have long-running conversations and have access to information that users want to know about.

Aside from basic prompting and LLMs, memory and retrieval are the core components of a chatbot. Memory allows a chatbot to remember past interactions, and retrieval provides a chatbot with up-to-date, domain-specific information.

In this project we will focus on memory.

You will find everything needed here : https://python.langchain.com/docs/modules/memory/ 

### Complete pre-requisite

#### Packages

In [8]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain.schema import SystemMessage
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory,
                              ConversationBufferWindowMemory,
                              ConversationKGMemory)
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


# You can add other package here if needed
    

#### Initialize LLM & ConversationChain

A ConversationChain is a type of chain that is specifically designed to handle conversations or dialogues, as opposed to individual inputs.

In [10]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain

llm = Cohere(max_tokens=512, cohere_api_key=os.environ.get('COHERE_API_KEY'))

# We will use a ConversationChain for our project
conversation = ConversationChain(llm=llm)
print(llm)


Cohere
Params: {'model': None, 'max_tokens': 512, 'temperature': 0.75, 'k': 0, 'p': 1, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'truncate': None}


## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.



### ConversationBufferMemory

The conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form.

In [15]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation_buffer_memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory=conversation_buffer_memory)

#### verification

In [17]:
conversation.invoke("What is the capital of France ?")

conversation.invoke("What is the capital of Spain ?")

conversation.invoke("What is the capital of Angola ?")

# We should be able to get the whole conversation inside the memory
print(conversation_buffer_memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What is the capital of France ?'), AIMessage(content=' The capital of France is Paris. It is located in the north-central part of the country, on the River Seine, and is one of the most populous and well-known cities in the world.\n\nParis has a rich history and culture and has been an important center of finance, commerce, fashion, art, and architecture. It is home to iconic landmarks such as the Eiffel Tower, the Louvre Museum, the Arc de Triomphe, and the Notre-Dame Cathedral.\n\nParis is known for its sophisticated cuisine, romantic ambiance, and lively atmosphere, making it a top tourist destination globally. Also, Paris is called the "City of Lights" and the "City of Love," adding to its allure.\n\nIf you\'d like more information on Paris or France, please ask; I\'d be happy to provide additional details! '), HumanMessage(content='What is the capital of France ?'), AIMessage(content=' As mentioned earlier, the capital

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- stocker les conversations passées, conserver les historiques des users
- permet d'affiner des requetes

##### Cons

- pas adapté pour des longs scénarios
- plus la taille augmente plus le buffer est moins performant



### ConversationSummaryMemory

The conversation summary memory creates a summary of the conversation over time. 

In [22]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

conversation_summary_memory = ConversationSummaryMemory(llm=llm)

conversation_buffer_memory = ConversationChain(llm=llm, memory=conversation_summary_memory)

#### Verification

In [25]:
conversation_buffer_memory.invoke("Gives me the historical chronology of France. I want as much dates and events as possibles.")

conversation_buffer_memory.invoke("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

conversation_buffer_memory.invoke("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

# We should be able to get the whole conversation inside the memory
print(conversation_buffer_memory.memory.buffer)

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised CohereAPIError: You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions.


 The human asks the AI to provide a comprehensive list of important dates and events throughout Spanish history. The AI lists the dates and events under each era: the Prehistoric Era, the Phoenician and Ancient Era, the Medieval Era, the Early Modern Era, and the Enlightenment Era. The AI then proceeds to do the same for German history, providing dates and events such as the migration of Germanic tribes, the division of the Carolingian Empire, and the effects of the Renaissance. 


#### From your observations - Gives the pros and cons of this memory 

##### Pros

- permet de résumer rapidement très généralement en gardant en memoire les échanges passés
- garde en memoire les infos importantes pour des requetes ultérieures

##### Cons

- pas adapté à trop de données et trop détaillées
- pas adapté à des echanges structurés / + organisés



### ConversationSummaryBufferMemory

The conversation summary buffer memory keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions, it compiles them into a summary and uses both.

In [27]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

conversation_summary_buffer_memory = ConversationSummaryBufferMemory(llm=llm)
# Initialize a ConversationChain with a memory in it
conversation = ConversationChain(llm=llm, memory=conversation_summary_buffer_memory)
    

#### Verification

In [28]:
conversation.invoke("Gives me the historical chronology of France. I want as much dates and events as possible.")

conversation.invoke("Gives me the historical chronology of France. I want as much dates and events as possibles.")

conversation.invoke("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

conversation.invoke("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

# We should be able to get the whole conversation inside the memory
print(conversation.memory.buffer)

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[HumanMessage(content='Gives me the historical chronology of France. I want as much dates and events as possibles.'), AIMessage(content=" Sure, here is a continuation of the list of important dates and events in French history:\n\n6. The French Wars of Religion (1562-1598) - The Wars unfolded between the Catholic majority and the Protestant Huguenots, over control of the French throne and religion in the country. The conflict concluded with the Edict of Nantes, which guaranteed religious liberties to Protestants and ended the Wars. \n\n7. The French Revolution (1789-1799) - Revolutionary fever seized France in 1789 with the outbreak of the Revolution. The monarchy was abolished, and sovereign power was vested in the hands of the people through the National Assembly. King Louis XVI was executed in 1793 after being condemned by the National Convention. The Revolution reached its climax with the rule of the Committee of Public Safety under the leadership of Robespierre during the Reign of

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- utilité pour analyser et synthétiser des conversations longues / complexes
- trace l'historique des conversations

##### Cons

- consomme bcp de memoire ?
- 

